# 5) Color Words & Description Density

**Goal:** Count color terms and compare description density.

# Setup: Load Texts

This notebook needs **Alice in Wonderland** and **Through the Looking-Glass** as input texts.

**How to provide the texts:**
1. Download books from Project Gutenberg (IDs 11 and 12) as txts. [go to https://www.gutenberg.org/ebooks/11 and https://www.gutenberg.org/ebooks/12]

2. Place two text files in the "data" folder with names:
   - `Wondeland.txt`  (Alice's Adventures in Wonderland)
   - `Looking-Glass.txt` (Through the Looking-Glass)

In [ ]:
import re
from pathlib import Path
from collections import Counter

In [ ]:

def load_texts(local_alice: str = '../data/Wonderland.txt',
               local_glass: str = '../data/Looking-Glass.txt'):
    """Load Wonderland and Looking-Glass texts from disk.

    Parameters
    ----------
    local_alice : str
        Path to Wonderland text file. Defaults to '../data/Wonderland.txt'.
    local_glass : str
        Path to Looking-Glass text file. Defaults to '../data/Looking-Glass.txt'.

    Returns
    -------
    tuple[str, str]
        (wonderland_text, lookingglass_text).

    Raises
    ------
    FileNotFoundError
        If either file is missing.

    Extra Notes
    -----------
    - Using UTF-8 with `errors='ignore'` avoids codec exceptions on
      older Project Gutenberg dumps or inconsistent encodings.
    """
    p1, p2 = Path(local_alice), Path(local_glass)

    # Fail fast with a clear message if a file is missing
    if not p1.exists():
        raise FileNotFoundError(
            f"Missing file: {p1}\n"
            "→ Please place 'Wonderland.txt' at this path or update load_texts(...)."
        )
    if not p2.exists():
        raise FileNotFoundError(
            f"Missing file: {p2}\n"
            "→ Please place 'Looking-Glass.txt' at this path or update load_texts(...)."
        )

    # Read the files (UTF-8; ignore undecodable bytes to stay robust)
    wonderland   = p1.read_text(encoding='utf-8', errors='ignore')
    lookingglass = p2.read_text(encoding='utf-8', errors='ignore')
    return wonderland, lookingglass

def normalize(text: str) -> str:
    """Normalize a Gutenberg-like text for tokenization.

    Steps
    -----
    1) Heuristically strip Project Gutenberg headers/footers if present
       (looks for *** START ... *** END markers).
    2) Normalize newlines to '\n'.

    Parameters
    ----------
    text : str
        Raw text as loaded from disk (can be empty).

    Returns
    -------
    str
        Cleaned text suitable for tokenization and counting.
    """
    if not text:
        return ''
    # Clip to the main body if markers are present.
    start = text.find('*** START')
    end   = text.find('*** END')
    if start != -1 and end != -1 and end > start:
        text = text[start:end]
    # Normalize Windows line endings.
    return text.replace('\r\n', '\n')

# Load raw texts (forgiving: returns '' if a file is missing)
wonderland_raw, lookingglass_raw = load_texts()

# Normalize for tokenization
wonderland   = normalize(wonderland_raw)
lookingglass = normalize(lookingglass_raw)

print(f"Wonderland chars: {len(wonderland):,} | Looking-Glass chars: {len(lookingglass):,}")


### Helpers: Tokenization

In [ ]:
WORD_RE = re.compile(r"[A-Za-z']+")  # keep apostrophes in words (e.g., don't -> don't)

def words(text: str):
    """Simple word tokenizer (lowercased, ASCII letters + apostrophes).

    Pros
    ----
    - Very fast and dependency-free.
    - Good enough for frequency/keyness demonstrations.

    Cons
    ----
    - No punctuation words, no sentence boundaries, no POS tags.
    - May treat possessives inconsistently across sources.

    Returns
    -------
    list[str]
        Lowercased word words.
    """
    return WORD_RE.findall(text.lower())


def sentences(text: str):
    """Naive sentence splitter using punctuation boundaries.

    Uses a regex to split on '.', '!', '?' followed by whitespace.
    Because this is heuristic, treat results as approximate.

    Returns
    -------
    list[str]
        Sentence-like strings.
    """
    return [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]



wonderland_words = words(wonderland)
lookingglass_words = words(lookingglass)

wonderland_sentences = sentences(wonderland)
lookingglass_sentences = sentences(lookingglass)

print(f"Wonderland words: {len(wonderland_words):,} | Looking-Glass words: {len(lookingglass_words):,}")
print(f"Wonderland sentences: {len(wonderland_sentences):,} | Looking Glass sentences: {len(lookingglass_sentences):,}")


### Count Color Terms

In [ ]:
COLOR_LIST = {
    'red','orange','yellow','green','blue','indigo','violet','purple','pink','brown','black','white','gray','grey',
    'scarlet','crimson','emerald','amber','gold','silver','lavender','mauve','ivory','beige','teal','turquoise','magenta','maroon','navy'
}
def count_colors(tokens):
    c = Counter(w for w in tokens if w in COLOR_LIST)
    return c, sum(c.values()), len(tokens)

a_c, a_hits, a_total = count_colors(wonderland_words)
g_c, g_hits, g_total = count_colors(lookingglass_words)
print("Alice top:", a_c.most_common(15), "| rate per 100k:", (a_hits/a_total)*100000)
print("Glass top:", g_c.most_common(15), "| rate per 100k:", (g_hits/g_total)*100000)


**Discuss:** Where do color bursts cluster in the narrative? What scenes rely on color to signal mood or magic?

In [ ]:
def rolling_color_windows(tokens, window=800, step=200, color_list=COLOR_LIST):
    hits_per_window = []
    for i in range(0, max(1, len(tokens)-window+1), step):
        chunk = tokens[i:i+window]
        c = sum(1 for w in chunk if w in color_list)
        hits_per_window.append((i, i+window, c, c * (100000/window)))  # per 100k
    return hits_per_window

def nearest_sentence_span(tokens, sents, start_idx, end_idx):
    # approximate: map token range to a sentence slice of similar length
    # (good enough for previewing passages)
    text = " ".join(tokens[start_idx:end_idx])
    # find a sentence that contains first few words of the window
    head = " ".join(tokens[start_idx:start_idx+20])
    for k, s in enumerate(sents):
        if head[:50] in s:
            j0 = max(0, k-1); j1 = min(len(sents), k+3)
            return j0, j1, " ".join(sents[j0:j1])
    return None, None, text[:500]

# run it
w_roll = rolling_color_windows(wonderland_words, window=800, step=200)
g_roll = rolling_color_windows(lookingglass_words, window=800, step=200)

# top 5 bursts by per-100k
w_top = sorted(w_roll, key=lambda x: x[3], reverse=True)[:5]
g_top = sorted(g_roll, key=lambda x: x[3], reverse=True)[:5]

print("=== Wonderland color bursts ===")
for a,b,h,rate in w_top:
    j0,j1,preview = nearest_sentence_span(wonderland_words, wonderland_sentences, a,b)
    print(f"[tokens {a}-{b}] hits={h} | {rate:.0f} per 100k")
    print(preview[:400], "…\n")

print("=== Looking-Glass color bursts ===")
for a,b,h,rate in g_top:
    j0,j1,preview = nearest_sentence_span(lookingglass_words, lookingglass_sentences, a,b)
    print(f"[tokens {a}-{b}] hits={h} | {rate:.0f} per 100k")
    print(preview[:400], "…\n")
